In [ ]:
# !pip install openai
# !pip install python-dotenv

In [34]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_type = "azure"
openai.api_base = "https://cog-gavkwsts6rr74openai.openai.azure.com/"
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_version = "2023-05-15"

In [44]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine="chat",
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a horror tone
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

In [45]:
response = get_completion(prompt)
response

'"By the depths of the abyss, I be seething with rage! The cursed lid of me blender did fly off and spatter me kitchen walls with a ghastly smoothie! And to add to the terror, the warranty be of no use in covering the cost of cleaning up this mess. I implore ye, lend me yer aid this instant, lest the spirits of this cursed kitchen haunt us forevermore!"'